## Data prep

In [7]:
import os
import time
import zipfile
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer

###############################################################################
# Utility functions

def extract_zip(zip_file, extract_to):
    """Extracts a zip file to the specified directory."""
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def getDataPoints(path):
    """Collects content of all .py files within the given directory."""
    dataPointsList = []
    if not os.path.exists(path):
        print(f"Directory does not exist: {path}")
        return dataPointsList

    for root, dirs, files in os.walk(path):
        for dataPointName in files:
            if dataPointName.endswith(".py"):
                file_path = os.path.join(root, dataPointName)
                with open(file_path, encoding="utf-8") as fileIn:
                    dp = fileIn.read().strip()
                    if dp:  # Ensure the document is not empty
                        dataPointsList.append(dp)
                    else:
                        print(f"Empty or invalid file skipped: {file_path}")

    if len(dataPointsList) == 0:
        print(f"No valid documents found in directory: {path}")

    return dataPointsList

###############################################################################
# Data Extraction and Vectorization

# Parameters setup
flakyZip = "Dataset/flaky_files.zip"
nonFlakyZip = "Dataset/all_nonflaky_files.zip"

outDir = "smote-results"
os.makedirs(outDir, exist_ok=True)
extractDir = "smote-extracted"
os.makedirs(extractDir, exist_ok=True)

# Extract the zip files
flakyDir = os.path.join(extractDir, 'flaky')
nonFlakyDir = os.path.join(extractDir, 'nonFlaky')
os.makedirs(flakyDir, exist_ok=True)
os.makedirs(nonFlakyDir, exist_ok=True)

extract_zip(flakyZip, flakyDir)
extract_zip(nonFlakyZip, nonFlakyDir)

dataPointsFlaky = getDataPoints(flakyDir)
dataPointsNonFlaky = getDataPoints(nonFlakyDir)
dataPoints = dataPointsFlaky + dataPointsNonFlaky

print(f"Number of flaky documents: {len(dataPointsFlaky)}")
print(f"Number of non-flaky documents: {len(dataPointsNonFlaky)}")
print(f"Total number of documents: {len(dataPoints)}")

if len(dataPoints) == 0:
    raise ValueError("No documents available for vectorization. Please check the input directories.")

# Create labels: 1 for flaky, 0 for non-flaky
dataLabelsList = np.array([1]*len(dataPointsFlaky) + [0]*len(dataPointsNonFlaky))

Number of flaky documents: 45
Number of non-flaky documents: 243
Total number of documents: 288


## KNN

In [8]:
import os
import time
import zipfile
import numpy as np
import pandas as pd
import math

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    precision_score, recall_score, accuracy_score, f1_score,
    matthews_corrcoef, make_scorer
)
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD  # Suitable for sparse data
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.decomposition import PCA

def runKNNWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits):
    # Define scoring metrics with MCC
    scoring = {
        'precision': make_scorer(precision_score, zero_division=1),
        'recall': make_scorer(recall_score, zero_division=1),
        'accuracy': make_scorer(accuracy_score),
        'f1': make_scorer(f1_score, zero_division=1),
        'mcc': make_scorer(matthews_corrcoef)  # Directly using matthews_corrcoef
        
    }

    # Define a pipeline with CountVectorizer, TruncatedSVD, SMOTE, and KNN
    pipeline = ImbPipeline([
        ('vectorizer', CountVectorizer(stop_words=None)),
        ('pca', PCA()),
        ('smote', SMOTE(random_state=42)),  # SMOTE for oversampling
        ('knn', KNeighborsClassifier()),  # KNN classifier
        

    ])

    # Define parameter grid for GridSearchCV
    param_grid = {
        'pca__n_components': [150,180,200,200],  # Variance ratios
        'knn__n_neighbors': [3, 5, 7, 9, 11, 15, 20],  # Number of neighbors for KNN
        'knn__weights': ['uniform', 'distance'],  # Uniform or distance-based weighting
        'knn__metric': ['euclidean', 'cosine']  # Distance metrics
    }

    # Setup cross-validation strategy
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Setup GridSearchCV with the pipeline and parameter grid
    grid_search = GridSearchCV(
        pipeline, param_grid, cv=skf, scoring=scoring,
        refit='f1', verbose=1, return_train_score=True
    )

    # Record the start time
    start_time = time.perf_counter()

    # Fit the GridSearchCV to the data
    grid_search.fit(dataPoints, dataLabelsList)

    # Record the end time
    end_time = time.perf_counter()
    total_time = end_time - start_time

    # Retrieve the best parameters and score from cross-validation
    best_params = grid_search.best_params_
    best_f1_cv = grid_search.best_score_

    print(f"Best Parameters with SMOTE and PCA: {best_params}")
    print(f"Best F1 Score from cross-validation: {best_f1_cv}")

    # Extract the cross-validation results and print final metrics
    results = grid_search.cv_results_

    # Prepare per-fold metrics for CSV along with the parameter combinations
    fold_metrics = []
    for idx in range(len(results['params'])):
        fold_metrics.append({
            'svd__n_components': results['params'][idx].get('svd__n_components'),
            'n_neighbors': results['params'][idx].get('knn__n_neighbors'),
            'weights': results['params'][idx].get('knn__weights'),
            'metric': results['params'][idx].get('knn__metric'),
            'accuracy': results['mean_test_accuracy'][idx],
            'precision': results['mean_test_precision'][idx],
            'recall': results['mean_test_recall'][idx],
            'f1': results['mean_test_f1'][idx],
            'mcc': results['mean_test_mcc'][idx],  # Include MCC in fold metrics
        })

    # Save fold-wise metrics to CSV
    df_folds = pd.DataFrame(fold_metrics)
    outFile_folds = os.path.join(outDir, f"knn-smote-pca-fold-results-{n_splits}-folds.csv")
    df_folds.to_csv(outFile_folds, index=False)

    print(f"Per-fold metrics saved to: {outFile_folds}")

    # Extract final metrics based on the cross-validation results
    best_index = grid_search.best_index_
    final_f1 = results['mean_test_f1'][best_index]
    final_precision = results['mean_test_precision'][best_index]
    final_recall = results['mean_test_recall'][best_index]
    final_accuracy = results['mean_test_accuracy'][best_index]
    final_mcc = results['mean_test_mcc'][best_index]  # Include final MCC

    # Print final metrics (cross-validation averages)
    print("\nFinal Cross-Validation Metrics:")
    print(f"Final Precision: {final_precision}")
    print(f"Final Recall: {final_recall}")
    print(f"Final Accuracy: {final_accuracy}")
    print(f"Final F1 Score: {final_f1}")
    print(f"Final MCC: {final_mcc}")  # Print final MCC

    # Save the final results to a CSV file
    outFile_final = os.path.join(outDir, f"knn-results-{n_splits}-folds.csv")
    with open(outFile_final, "w") as f:
        f.write("Accuracy,Precision,Recall,F1,MCC\n")  # Include MCC in header
        f.write(f"{final_accuracy},{final_precision},{final_recall},{final_f1},{final_mcc}\n")  # Include MCC

    print(f"KNN analysis completed for {n_splits}-folds with SMOTE and PCA. Results saved to: {outFile_final}")

    return best_params, final_f1, final_mcc  # Return final MCC as well




outDir = "smote-results"
os.makedirs(outDir, exist_ok=True)
extractDir = "smote-extracted"
os.makedirs(extractDir, exist_ok=True)  # Corrected 'Tarue' to 'True'

# Extract the zip files
flakyDir = os.path.join(extractDir, 'flaky')
nonFlakyDir = os.path.join(extractDir, 'nonFlaky')
os.makedirs(flakyDir, exist_ok=True)
os.makedirs(nonFlakyDir, exist_ok=True)

extract_zip(flakyZip, flakyDir)
extract_zip(nonFlakyZip, nonFlakyDir)

dataPointsFlaky = getDataPoints(flakyDir)
dataPointsNonFlaky = getDataPoints(nonFlakyDir)
dataPoints = dataPointsFlaky + dataPointsNonFlaky

print(f"Number of flaky documents: {len(dataPointsFlaky)}")
print(f"Number of non-flaky documents: {len(dataPointsNonFlaky)}")
print(f"Total number of documents: {len(dataPoints)}")

if len(dataPoints) == 0:
    raise ValueError("No documents available for vectorization. Please check the input directories.")

# Create labels: 1 for flaky, 0 for non-flaky
dataLabelsList = np.array([1]*len(dataPointsFlaky) + [0]*len(dataPointsNonFlaky))





# Run KNN with SMOTE and PCA using 5-fold cross-validation
print("\nStarting KNN analysis with SMOTE and PCA for 5-fold cross-validation...")
best_params_5folds, best_f1_5folds, final_mcc_5folds = runKNNWithSMOTE(
    dataPoints, dataLabelsList, outDir, n_splits=5
)


print("\nBest results for KNN with SMOTE and PCA 5-fold cross-validation:")
print(f"Best Parameters: {best_params_5folds}")
print(f"Best F1 Score: {best_f1_5folds}")
print(f"Best MCC: {final_mcc_5folds}")


Number of flaky documents: 45
Number of non-flaky documents: 243
Total number of documents: 288

Starting KNN analysis with SMOTE and PCA for 5-fold cross-validation...
Fitting 5 folds for each of 112 candidates, totalling 560 fits
Best Parameters with SMOTE and PCA: {'knn__metric': 'cosine', 'knn__n_neighbors': 15, 'knn__weights': 'distance', 'pca__n_components': 200}
Best F1 Score from cross-validation: 0.6066511266511266
Per-fold metrics saved to: smote-results\knn-smote-pca-fold-results-5-folds.csv

Final Cross-Validation Metrics:
Final Precision: 0.6165112665112665
Final Recall: 0.6444444444444445
Final Accuracy: 0.8785238959467634
Final F1 Score: 0.6066511266511266
Final MCC: 0.5512693059318502
KNN analysis completed for 5-folds with SMOTE and PCA. Results saved to: smote-results\knn-results-5-folds.csv

Best results for KNN with SMOTE and PCA 5-fold cross-validation:
Best Parameters: {'knn__metric': 'cosine', 'knn__n_neighbors': 15, 'knn__weights': 'distance', 'pca__n_components

## SVM

In [13]:
import numpy as np
import pandas as pd

from sklearn.svm import SVC
from sklearn.metrics import (
    precision_score, recall_score, accuracy_score, f1_score
)
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

def runSVMWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits):
    v0 = time.perf_counter()

    # Define scoring metrics
    scoring = {
        'precision': 'precision',
        'recall': 'recall',
        'accuracy': 'accuracy',
        'f1': 'f1',
        'mcc': make_scorer(matthews_corrcoef)  

        
    }

    # Define a pipeline with SMOTE and SVM 
    pipeline = ImbPipeline([
        ('vectorizer', CountVectorizer(stop_words=None)),  # Include vectorizer in pipeline
        ('pca', PCA()),
        ('smote', SMOTE(random_state=42)),    # SMOTE for oversampling
        ('svm', SVC(probability=True))        # SVM classifier
    ])

    # Define parameter grid for GridSearchCV
    param_grid = {
    'pca__n_components': [150, 180, 200, 220],           # PCA components
    'svm__C': [0.01, 0.1, 1.0, 10.0, 100.0],            # Regularization parameter C for SVM
    'svm__kernel': ['linear', 'rbf', 'poly', 'sigmoid']  # Kernel types for SVM
        }

    # Setup cross-validation strategy
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Setup GridSearchCV with the pipeline and parameter grid
    grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring=scoring, refit='f1', verbose=1, return_train_score=True)

    # Fit the GridSearchCV to the data
    grid_search.fit(dataPoints, dataLabelsList)

    # Retrieve the best parameters and score from cross-validation
    best_params = grid_search.best_params_
    best_f1_cv = grid_search.best_score_

    print(f"Best Parameters with SMOTE: {best_params}")
    print(f"Best F1 Score from cross-validation: {best_f1_cv}")

    # Extract the cross-validation results and print final metrics
    results = grid_search.cv_results_

    # Prepare per-fold metrics for CSV along with the parameter combinations
    fold_metrics = []
    for idx in range(len(results['params'])):
        fold_metrics.append({
            'C': results['params'][idx].get('svm__C'),
            'kernel': results['params'][idx].get('svm__kernel'),
            'accuracy': results['mean_test_accuracy'][idx],
            'precision': results['mean_test_precision'][idx],
            'recall': results['mean_test_recall'][idx],
            'f1': results['mean_test_f1'][idx],
            'mcc': results['mean_test_mcc'][idx],
        })

    # Save fold-wise metrics to CSV
    df_folds = pd.DataFrame(fold_metrics)
    outFile = os.path.join(outDir, f"svm-smote-fold-results-{n_splits}-folds.csv")
    df_folds.to_csv(outFile, index=False)

    print(f"Per-fold metrics saved to: {outFile}")

    # Extract final metrics based on the cross-validation results
    final_f1 = results['mean_test_f1'][grid_search.best_index_]
    final_precision = results['mean_test_precision'][grid_search.best_index_]
    final_recall = results['mean_test_recall'][grid_search.best_index_]
    final_accuracy = results['mean_test_accuracy'][grid_search.best_index_]
    final_mcc = results['mean_test_mcc'][grid_search.best_index_]

    # Print final metrics (cross-validation averages)
    print("\nFinal Cross-Validation Metrics:")
    print(f"Final Precision: {final_precision}")
    print(f"Final Recall: {final_recall}")
    print(f"Final Accuracy: {final_accuracy}")
    print(f"Final F1 Score: {final_f1}")

    # Save the results to a CSV file
    outFile = os.path.join(outDir, f"svm-results-{n_splits}-folds.csv")
    with open(outFile, "w") as f:
        f.write("Accuracy,Precision,Recall,F1\n")
        f.write(f"{final_accuracy},{final_precision},{final_recall},{final_f1}\n")

    print(f"SVM analysis completed for {n_splits}-folds with SMOTE. Results saved to: {outFile}")

    return best_params, final_f1



# Run SVM with SMOTE using 5-fold cross-validation
print("\nStarting SVM analysis with SMOTE for 5-fold cross-validation...")
best_params_5folds, best_f1_5folds = runSVMWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits=5)


print("\nBest results for SVM with SMOTE 5-fold cross-validation:")
print(f"Best Parameters: {best_params_5folds}")
print(f"Best F1 Score: {best_f1_5folds}")



Starting SVM analysis with SMOTE for 5-fold cross-validation...
Fitting 5 folds for each of 80 candidates, totalling 400 fits


C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\metrics\_clas

C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\haha9\anaconda3\Lib\site-packages\sklearn\metrics\_clas

Best Parameters with SMOTE: {'pca__n_components': 220, 'svm__C': 0.01, 'svm__kernel': 'linear'}
Best F1 Score from cross-validation: 0.6914602683178535
Per-fold metrics saved to: smote-results\svm-smote-fold-results-5-folds.csv

Final Cross-Validation Metrics:
Final Precision: 0.8445238095238097
Final Recall: 0.6222222222222222
Final Accuracy: 0.9201451905626135
Final F1 Score: 0.6914602683178535
SVM analysis completed for 5-folds with SMOTE. Results saved to: smote-results\svm-results-5-folds.csv

Best results for SVM with SMOTE 5-fold cross-validation:
Best Parameters: {'pca__n_components': 220, 'svm__C': 0.01, 'svm__kernel': 'linear'}
Best F1 Score: 0.6914602683178535


## XGB

In [18]:
import os
import time
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import (precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef, make_scorer)
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer

###############################################################################
# Custom MCC scorer function
def mcc_scorer(estimator, X, y_true):
    y_pred = estimator.predict(X)
    return matthews_corrcoef(y_true, y_pred)

###############################################################################
# XGBoost with SMOTE, PCA, and CountVectorizer

def runXGBWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits):
    v0 = time.perf_counter()

    # Define scoring metrics
    scoring = {
        'precision': make_scorer(precision_score, zero_division=1),
        'recall': make_scorer(recall_score, zero_division=1),
        'accuracy': make_scorer(accuracy_score),
        'f1': make_scorer(f1_score, zero_division=1),
        'mcc': mcc_scorer  # Add MCC to scoring
    }

    # Define a pipeline with CountVectorizer, PCA, SMOTE, and XGBoost
    pipeline = ImbPipeline([
        ('vectorizer', CountVectorizer(stop_words=None)),  
        ('pca', PCA()),                                    
        ('smote', SMOTE(random_state=42)),                 
        ('xgb', XGBClassifier(eval_metric='logloss', random_state=42))  
    ])

    # Define parameter grid for GridSearchCV
    param_grid = {
        'pca__n_components': [150,180, 200, 220],             
        'xgb__n_estimators': [100, 150, 200],         
        'xgb__max_depth': [3, 5, 7, 10],                  
        'xgb__learning_rate': [0.01, 0.1, 0.3, 0.5],      
    }

    # Setup cross-validation strategy
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Setup GridSearchCV with the pipeline and parameter grid
    grid_search = GridSearchCV(
        pipeline, param_grid, cv=skf, scoring=scoring, refit='f1', verbose=1, return_train_score=True
    )

    # Fit the GridSearchCV to the data
    grid_search.fit(dataPoints, dataLabelsList)

    # Retrieve the best parameters and score from cross-validation
    best_params = grid_search.best_params_
    best_f1_cv = grid_search.best_score_

    print(f"Best Parameters with SMOTE, PCA, and CountVectorizer: {best_params}")
    print(f"Best F1 Score from cross-validation: {best_f1_cv}")

    # Extract the cross-validation results and print final metrics
    results = grid_search.cv_results_

    # Prepare per-fold metrics for CSV along with the parameter combination
    fold_metrics = []
    for idx in range(len(results['params'])):
        fold_metrics.append({
            'n_components': results['params'][idx].get('pca__n_components'),
            'n_estimators': results['params'][idx].get('xgb__n_estimators'),
            'max_depth': results['params'][idx].get('xgb__max_depth'),
            'learning_rate': results['params'][idx].get('xgb__learning_rate'),
            'accuracy': results.get('mean_test_accuracy', [None])[idx],
            'precision': results.get('mean_test_precision', [None])[idx],
            'recall': results.get('mean_test_recall', [None])[idx],
            'f1': results.get('mean_test_f1', [None])[idx],
            'mcc': results.get('mean_test_mcc', [None])[idx],  # Add MCC to fold metrics
        })

    # Save fold-wise metrics to CSV
    df_folds = pd.DataFrame(fold_metrics)
    outFile_folds = os.path.join(outDir, f"xgb-smote-pca-fold-results-{n_splits}-folds.csv")
    df_folds.to_csv(outFile_folds, index=False)

    print(f"Per-fold metrics saved to: {outFile_folds}")

    # Extract final metrics based on the cross-validation results
    final_f1 = results['mean_test_f1'][grid_search.best_index_]
    final_precision = results['mean_test_precision'][grid_search.best_index_]
    final_recall = results['mean_test_recall'][grid_search.best_index_]
    final_accuracy = results['mean_test_accuracy'][grid_search.best_index_]
    final_mcc = results['mean_test_mcc'][grid_search.best_index_]  # Extract final MCC

    # Print final metrics (cross-validation averages)
    print("\nFinal Cross-Validation Metrics:")
    print(f"Final Precision: {final_precision}")
    print(f"Final Recall: {final_recall}")
    print(f"Final Accuracy: {final_accuracy}")
    print(f"Final F1 Score: {final_f1}")
    print(f"Final MCC: {final_mcc}")  # Print final MCC

    # Save the results to a CSV file
    outFile_final = os.path.join(outDir, f"xgb-results-{n_splits}-folds.csv")
    with open(outFile_final, "w") as f:
        f.write("Accuracy,Precision,Recall,F1,MCC\n")  # Include MCC in the header
        f.write(f"{final_accuracy},{final_precision},{final_recall},{final_f1},{final_mcc}\n")  # Include MCC in the data

    print(f"XGBoost analysis completed for {n_splits}-folds with SMOTE, PCA, and CountVectorizer. Results saved to: {outFile_final}")

    return best_params, final_f1, final_mcc  # Return final MCC

###############################################################################
# Main Execution for 5-Fold Cross-Validation

outDir = "smote-results"
os.makedirs(outDir, exist_ok=True)

# Run XGBoost with SMOTE, PCA, and CountVectorizer using 5-fold cross-validation
print("\nStarting XGBoost analysis with SMOTE, PCA, and CountVectorizer for 5-fold cross-validation...")
best_params_5folds, best_f1_5folds, final_mcc_5folds = runXGBWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits=5)

# Display results
print("\nBest results for XGBoost with SMOTE, PCA, and CountVectorizer 5-fold cross-validation:")
print(f"Best Parameters: {best_params_5folds}")
print(f"Best F1 Score: {best_f1_5folds}")
print(f"Final MCC: {final_mcc_5folds}")



Starting XGBoost analysis with SMOTE, PCA, and CountVectorizer for 5-fold cross-validation...
Fitting 5 folds for each of 192 candidates, totalling 960 fits


C:\Users\haha9\anaconda3\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best Parameters with SMOTE, PCA, and CountVectorizer: {'pca__n_components': 150, 'xgb__learning_rate': 0.1, 'xgb__max_depth': 3, 'xgb__n_estimators': 150}
Best F1 Score from cross-validation: 0.6997435897435896
Per-fold metrics saved to: smote-results\xgb-smote-pca-fold-results-5-folds.csv

Final Cross-Validation Metrics:
Final Precision: 0.8714285714285716
Final Recall: 0.6
Final Accuracy: 0.920205686630369
Final F1 Score: 0.6997435897435896
Final MCC: 0.677301901085489
XGBoost analysis completed for 5-folds with SMOTE, PCA, and CountVectorizer. Results saved to: smote-results\xgb-results-5-folds.csv

Best results for XGBoost with SMOTE, PCA, and CountVectorizer 5-fold cross-validation:
Best Parameters: {'pca__n_components': 150, 'xgb__learning_rate': 0.1, 'xgb__max_depth': 3, 'xgb__n_estimators': 150}
Best F1 Score: 0.6997435897435896
Final MCC: 0.677301901085489


## Random Forest

In [12]:
import os
import time
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef, make_scorer)
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer

###############################################################################
# Custom MCC scorer function
def mcc_scorer(estimator, X, y_true):
    y_pred = estimator.predict(X)
    return matthews_corrcoef(y_true, y_pred)

###############################################################################
# Random Forest with SMOTE and PCA

def runRFWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits):
    v0 = time.perf_counter()

    # Define scoring metrics
    scoring = {
        'precision': make_scorer(precision_score, zero_division=1),
        'recall': make_scorer(recall_score, zero_division=1),
        'accuracy': make_scorer(accuracy_score),
        'f1': make_scorer(f1_score, zero_division=1),
        'mcc': mcc_scorer  # Add MCC to scoring
    }

    # Define a pipeline with PCA, SMOTE, and Random Forest
    pipeline = ImbPipeline([
        ('vectorizer', CountVectorizer(stop_words=None)), 
        ('smote', SMOTE(random_state=42)),                
        ('rf', RandomForestClassifier(random_state=42))  
    ])

    # Define parameter grid for GridSearchCV
    param_grid = {
        'rf__n_estimators': [50,100,200],  
        'rf__criterion': ['gini', 'entropy'],   
        'rf__max_depth': [10, 20,30],  
        'rf__min_samples_split': [5, 10],       
        'rf__min_samples_leaf': [2, 5],        
    }

    # Setup cross-validation strategy
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Setup GridSearchCV with the pipeline and parameter grid
    grid_search = GridSearchCV(
        pipeline, param_grid, cv=skf, scoring=scoring, refit='f1', verbose=1, return_train_score=True
    )

    # Fit the GridSearchCV to the data
    grid_search.fit(dataPoints, dataLabelsList)

    # Retrieve the best parameters and score from cross-validation
    best_params = grid_search.best_params_
    best_f1_cv = grid_search.best_score_

    print(f"Best Parameters with SMOTE and PCA: {best_params}")
    print(f"Best F1 Score from cross-validation: {best_f1_cv}")

    # Extract the cross-validation results and print final metrics
    results = grid_search.cv_results_

    # Prepare per-fold metrics for CSV along with the parameter combinations
    fold_metrics = []
    for idx in range(len(results['params'])):
        fold_metrics.append({
            'n_components': results['params'][idx].get('pca__n_components'),
            'n_estimators': results['params'][idx].get('rf__n_estimators'),
            'max_depth': results['params'][idx].get('rf__max_depth'),
            'min_samples_split': results['params'][idx].get('rf__min_samples_split'),
            'min_samples_leaf': results['params'][idx].get('rf__min_samples_leaf'),
            'criterion': results['params'][idx].get('rf__criterion'),
            'accuracy': results.get('mean_test_accuracy', [None])[idx],  # Using dynamic access to avoid KeyError
            'precision': results.get('mean_test_precision', [None])[idx],
            'recall': results.get('mean_test_recall', [None])[idx],
            'f1': results.get('mean_test_f1', [None])[idx],
            'mcc': results.get('mean_test_mcc', [None])[idx],  # Add MCC to fold metrics
        })

    # Save fold-wise metrics to CSV
    df_folds = pd.DataFrame(fold_metrics)
    outFile_folds = os.path.join(outDir, f"rf-smote-pca-fold-results-{n_splits}-folds.csv")
    df_folds.to_csv(outFile_folds, index=False)

    print(f"Per-fold metrics saved to: {outFile_folds}")

    # Extract final metrics based on the cross-validation results
    final_f1 = results['mean_test_f1'][grid_search.best_index_]
    final_precision = results['mean_test_precision'][grid_search.best_index_]
    final_recall = results['mean_test_recall'][grid_search.best_index_]
    final_accuracy = results['mean_test_accuracy'][grid_search.best_index_]
    final_mcc = results['mean_test_mcc'][grid_search.best_index_]  # Extract final MCC

    # Print final metrics (cross-validation averages)
    print("\nFinal Cross-Validation Metrics:")
    print(f"Final Precision: {final_precision}")
    print(f"Final Recall: {final_recall}")
    print(f"Final Accuracy: {final_accuracy}")
    print(f"Final F1 Score: {final_f1}")
    print(f"Final MCC: {final_mcc}")  # Print final MCC

    # Save the results to a CSV file
    outFile_final = os.path.join(outDir, f"rf-results-{n_splits}-folds.csv")
    with open(outFile_final, "w") as f:
        f.write("Accuracy,Precision,Recall,F1,MCC\n")  # Include MCC in header
        f.write(f"{final_accuracy},{final_precision},{final_recall},{final_f1},{final_mcc}\n")  # Include MCC in data

    print(f"Random Forest analysis completed for {n_splits}-folds with SMOTE and PCA. Results saved to: {outFile_final}")

    return best_params, final_f1, final_mcc  # Return final MCC

###############################################################################
# Main Execution for 5-Fold Cross-Validation

outDir = "smote-results"
os.makedirs(outDir, exist_ok=True)

# Run Random Forest with SMOTE and PCA using 5-fold cross-validation
print("\nStarting Random Forest analysis with SMOTE and PCA for 5-fold cross-validation...")
best_params_5folds, best_f1_5folds, final_mcc_5folds = runRFWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits=5)

# Display results
print("\nBest results for Random Forest with SMOTE and PCA 5-fold cross-validation:")
print(f"Best Parameters: {best_params_5folds}")
print(f"Best F1 Score: {best_f1_5folds}")
print(f"Final MCC: {final_mcc_5folds}")



Starting Random Forest analysis with SMOTE and PCA for 5-fold cross-validation...
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best Parameters with SMOTE and PCA: {'rf__criterion': 'entropy', 'rf__max_depth': 20, 'rf__min_samples_leaf': 2, 'rf__min_samples_split': 5, 'rf__n_estimators': 100}
Best F1 Score from cross-validation: 0.822015823873409
Per-fold metrics saved to: smote-results\rf-smote-pca-fold-results-5-folds.csv

Final Cross-Validation Metrics:
Final Precision: 0.8238888888888889
Final Recall: 0.8222222222222222
Final Accuracy: 0.9444041137326075
Final F1 Score: 0.822015823873409
Final MCC: 0.7898138504372108
Random Forest analysis completed for 5-folds with SMOTE and PCA. Results saved to: smote-results\rf-results-5-folds.csv

Best results for Random Forest with SMOTE and PCA 5-fold cross-validation:
Best Parameters: {'rf__criterion': 'entropy', 'rf__max_depth': 20, 'rf__min_samples_leaf': 2, 'rf__min_samples_split': 5, 'rf__n_estimators': 100}
Best F1 Sco

## Decision Tree

In [17]:
import os
import time
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef, make_scorer)
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer


def mcc_scorer(estimator, X, y_true):
    y_pred = estimator.predict(X)
    return matthews_corrcoef(y_true, y_pred)


def runDTWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits):
    v0 = time.perf_counter()

    # Define scoring metrics
    scoring = {
        'precision': make_scorer(precision_score, zero_division=1),
        'recall': make_scorer(recall_score, zero_division=1),
        'accuracy': make_scorer(accuracy_score),
        'f1': make_scorer(f1_score, zero_division=1),
        'mcc': mcc_scorer  # Add MCC to scoring
    }

    # Define a pipeline with PCA, SMOTE, and Decision Tree
    pipeline = ImbPipeline([
        ('vectorizer', CountVectorizer(stop_words=None)),  # Include vectorizer in pipeline
        ('smote', SMOTE(random_state=42)),                 # SMOTE for oversampling
        ('dt', DecisionTreeClassifier(random_state=42))    # Decision Tree classifier
    ])

    param_grid = {
        'dt__criterion': ['gini', 'entropy'], 
        'dt__max_depth': [10, 20,30, ],  
        'dt__min_samples_split': [5, 10], 
        'dt__min_samples_leaf': [ 2, 5],  
        'dt__max_features': [ 'sqrt', 'log2'],  
    }

    # Setup cross-validation strategy
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Setup GridSearchCV with the pipeline and parameter grid
    grid_search = GridSearchCV(
        pipeline, param_grid, cv=skf, scoring=scoring,
        refit='f1', verbose=1, return_train_score=True
    )

    # Fit the GridSearchCV to the data
    grid_search.fit(dataPoints, dataLabelsList)

    # Retrieve the best parameters and score from cross-validation
    best_params = grid_search.best_params_
    best_f1_cv = grid_search.best_score_

    print(f"Best Parameters with SMOTE and PCA: {best_params}")
    print(f"Best F1 Score from cross-validation: {best_f1_cv}")

    # Extract the cross-validation results and print final metrics
    results = grid_search.cv_results_

    # Prepare per-fold metrics for CSV along with the parameter combinations
    fold_metrics = []
    for idx in range(len(results['params'])):
        fold_metrics.append({
            'max_depth': results['params'][idx].get('dt__max_depth'),
            'min_samples_split': results['params'][idx].get('dt__min_samples_split'),
            'min_samples_leaf': results['params'][idx].get('dt__min_samples_leaf'),
            'criterion': results['params'][idx].get('dt__criterion'),
            'accuracy': results.get('mean_test_accuracy', [None])[idx],  # Using dynamic access to avoid KeyError
            'precision': results.get('mean_test_precision', [None])[idx],
            'recall': results.get('mean_test_recall', [None])[idx],
            'f1': results.get('mean_test_f1', [None])[idx],
            'mcc': results.get('mean_test_mcc', [None])[idx],  # Add MCC to fold metrics
        })

    # Save fold-wise metrics to CSV
    df_folds = pd.DataFrame(fold_metrics)
    outFile_folds = os.path.join(outDir, f"dt-smote-pca-fold-results-{n_splits}-folds.csv")
    df_folds.to_csv(outFile_folds, index=False)

    print(f"Per-fold metrics saved to: {outFile_folds}")

    # Extract final metrics based on the cross-validation results
    final_f1 = results['mean_test_f1'][grid_search.best_index_]
    final_precision = results['mean_test_precision'][grid_search.best_index_]
    final_recall = results['mean_test_recall'][grid_search.best_index_]
    final_accuracy = results['mean_test_accuracy'][grid_search.best_index_]
    final_mcc = results['mean_test_mcc'][grid_search.best_index_]  # Extract final MCC

    # Print final metrics (cross-validation averages)
    print("\nFinal Cross-Validation Metrics:")
    print(f"Final Precision: {final_precision}")
    print(f"Final Recall: {final_recall}")
    print(f"Final Accuracy: {final_accuracy}")
    print(f"Final F1 Score: {final_f1}")
    print(f"Final MCC: {final_mcc}")  # Print final MCC

    # Save the results to a CSV file
    outFile_final = os.path.join(outDir, f"dt-results-{n_splits}-folds.csv")
    with open(outFile_final, "w") as f:
        f.write("Accuracy,Precision,Recall,F1,MCC\n")  # Include MCC in header
        f.write(f"{final_accuracy},{final_precision},{final_recall},{final_f1},{final_mcc}\n")  # Include MCC in data

    print(f"Decision Tree analysis completed for {n_splits}-folds with SMOTE and PCA. Results saved to: {outFile_final}")

    return best_params, final_f1, final_mcc  # Return final MCC

###############################################################################
# Main Execution for 5-Fold Cross-Validation

outDir = "smote-results"
os.makedirs(outDir, exist_ok=True)

# Run Decision Tree with SMOTE and PCA using 5-fold cross-validation
print("\nStarting Decision Tree analysis with SMOTE and PCA for 5-fold cross-validation...")
best_params_5folds, best_f1_5folds, final_mcc_5folds = runDTWithSMOTE(dataPoints, dataLabelsList, outDir, n_splits=5)

# Display results
print("\nBest results for Decision Tree with SMOTE and PCA 5-fold cross-validation:")
print(f"Best Parameters: {best_params_5folds}")
print(f"Best F1 Score: {best_f1_5folds}")
print(f"Final MCC: {final_mcc_5folds}")



Starting Decision Tree analysis with SMOTE and PCA for 5-fold cross-validation...
Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best Parameters with SMOTE and PCA: {'dt__criterion': 'entropy', 'dt__max_depth': 10, 'dt__max_features': 'sqrt', 'dt__min_samples_leaf': 2, 'dt__min_samples_split': 5}
Best F1 Score from cross-validation: 0.6354264235338405
Per-fold metrics saved to: smote-results\dt-smote-pca-fold-results-5-folds.csv

Final Cross-Validation Metrics:
Final Precision: 0.6124542124542125
Final Recall: 0.6888888888888889
Final Accuracy: 0.8716878402903812
Final F1 Score: 0.6354264235338405
Final MCC: 0.569608383914997
Decision Tree analysis completed for 5-folds with SMOTE and PCA. Results saved to: smote-results\dt-results-5-folds.csv

Best results for Decision Tree with SMOTE and PCA 5-fold cross-validation:
Best Parameters: {'dt__criterion': 'entropy', 'dt__max_depth': 10, 'dt__max_features': 'sqrt', 'dt__min_samples_leaf': 2, 'dt__min_samples_split': 5}
Best

In [ ]:
import os
import pandas as pd

# Function to extract the best results from the CSV files of each model
def extract_best_results(model_name, fold, csv_file):
    """
    Extracts the best result from the CSV file for a model.

    Parameters:
    - model_name: The name of the model (e.g., "XGBoost", "Random Forest", "Decision Tree")
    - fold: Number of folds (e.g., 5 or 3)
    - csv_file: The path to the CSV file containing the model's results

    Returns:
    A dictionary containing the best results for the model and fold.
    """
    if not os.path.exists(csv_file):
        print(f"CSV file for {model_name} ({fold}-fold) does not exist: {csv_file}")
        return None

    # Read the CSV file
    df = pd.read_csv(csv_file)

    # Get the row with the best F1 score
    best_row = df.loc[df['f1'].idxmax()]

    # Collect the best results into a dictionary
    best_results = {
        'Model': model_name,
        'Fold': f"{fold}-fold",
        'Best Accuracy': best_row['accuracy'],
        'Best Precision': best_row['precision'],
        'Best Recall': best_row['recall'],
        'Best F1 Score': best_row['f1'],
        'Best Parameters': best_row.to_dict()  # Including all parameters
    }

    return best_results

# Function to gather and print/save the best results from all SMOTE models
def gather_best_results(models_results_dir, output_file):
    """
    Gathers the best results from all SMOTE models and writes them to a CSV file.

    Parameters:
    - models_results_dir: Directory where the model result CSV files are stored for the SMOTE models.
    - output_file: Path to the output CSV file to store the best results.
    """
    models = {
        'KNN': {'5-fold': 'knn-smote-fold-results-5-folds.csv'},
        'SVM': {'5-fold': 'svm-smote-fold-results-5-folds.csv'},
        'Naive Bayes': {'5-fold': 'nb-smote-fold-results-5-folds.csv'},
        'XGBoost': {'5-fold': 'xgb-smote-fold-results-5-folds.csv'},
        'Random Forest': {'5-fold': 'rf-smote-fold-results-5-folds.csv'},
        'Decision Tree': {'5-fold': 'dt-smote-fold-results-5-folds.csv',}
    }

    # Initialize an empty list to store the best results from each model and fold
    best_results = []

    for model_name, folds in models.items():
        for fold, csv_file in folds.items():
            full_csv_path = os.path.join(models_results_dir, csv_file)
            best_result = extract_best_results(model_name, fold, full_csv_path)
            if best_result:
                best_results.append(best_result)

    # Convert the list of best results into a DataFrame
    best_results_df = pd.DataFrame(best_results)

    # Save the best results to the output CSV file
    best_results_df.to_csv(output_file, index=False)
    print(f"Best results saved to: {output_file}")

    # Print the best results as a table
    print(f"\nBest Results from All SMOTE Models")
    print(best_results_df.to_string(index=False))

# Example usage
if __name__ == "__main__":
    # Directories where the model result CSV files are stored
    results_dir = 'smote-results'

    # Path to the output CSV file where best results will be stored
    output_file = "best_results_smote.csv"

    # Gather and save the best results
    gather_best_results(results_dir, output_file)
